In [14]:
import numpy as np
import glob
import sys, dill
sys.path.append('../src')
from lib.utils import *

In [23]:
x_scaler.min = x_scaler.min[:-1]
x_scaler.max = x_scaler.max[:-1]

In [24]:
def save_object(obj, filename):
    dill.dump(obj, file=open(filename, "wb"))

In [26]:
save_object(x_scaler, 'x_scaler.dill')

In [27]:
save_object(y_scaler, 'y_scaler.dill')

In [16]:
x_scaler = dill.load(open(f"x_minmax_scaler.pkl", 'rb'))
y_scaler = dill.load(open(f"y_minmax_scaler.pkl", 'rb'))

In [32]:
def sort_fn(filename):
    date_string = filename[-14:-4]
    datetime_object = datetime.datetime.strptime(date_string, "%Y_%m_%d")
    return datetime_object

x_path = sorted(glob.glob("../CAM5/inputs*"), key=sort_fn)
y_path = sorted(glob.glob("../CAM5/outputs*"), key=sort_fn)

In [36]:
x = np.load(x_path[0])[:, :-1]
x = x_scaler.transform(x)

In [40]:
x.max()

1.5042228

In [22]:
x_scaler.min.shape

(109,)

In [21]:
y_scaler.min.shape

(112,)